## LLM Function calling (tool use)


**Function calling** is the ability of a LLM to interact with customer code / external services. *Tool use* is often used to refer to the same concept. This is often a fine-tuned feature, thus not available to all models.

#### Why function call?
- keep responses up-to-date
- off-loads math,retrieval etc. tasks that are not stable with LLM hallucination
- grounding to verifiable data

In [ ]:
# Setup environment
import json, pandas as pd, yfinance as yf

## Defining functions for LLM use

### OpenAI Function Schema

[OpenAI Function Calling Documentation](https://platform.openai.com/docs/guides/function-calling)

Functions can be set in the tools parameter of each API request.

A function is defined by its schema, informing LLM about what it does (for LLM to determine whether to use) and what input arguments it expects. The schema contains following fields:

| Field         | Description                                                  |
| :------------ | :----------------------------------------------------------- |
| `type`        | This should always be `function`                             |
| `name`        | The function's name (e.g. `fetch_stock_data`)                |
| `description` | Details on when and how to use the function                  |
| `parameters`  | [JSON schema](https://json-schema.org/) defining the function's input arguments |
| `strict`      | Whether to enforce strict mode for the function call         |

In [69]:
example_tools = [
    # Tool 1: Fetch stock data
    {
        "type": "function",
        "function":{
            "name": "fetch_stock_highest_price",
            "description": "Fetch historical stock data for a given ticker between start_date and end_date and return the highest stock price",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Stock ticker symbol"
                    },
                    "start_date": {
                        "type": "string",
                        "format": "date",
                        "description": "Start date in 'YYYY-MM-DD' format"
                    },
                    "end_date": {
                        "type": "string",
                        "format": "date",
                        "description": "End date in 'YYYY-MM-DD' format"
                    }
                },
                "required": ["ticker", "start_date", "end_date"]
            }
        }
    },
    # Tool 2: Get exchange rate
    {
        "type": "function",
        "function":{
            "name": "get_exchange_rate",
            "description": "Get the current exchange rate of a base currency and target currency",
            "parameters": {
                "type": "object",
                "properties": {
                    "base_currency": {
                        "type": "string",
                        "description": "The base currency for exchange rate calculations, i.e. USD, EUR, CNY"
                    },
                    "target_currency": {
                        "type": "string",
                        "description": "The target currency for exchange rate calculations, i.e. USD, EUR, CNY"
                    }
                },
                "required": ["base_currency", "target_currency"]
            }
        }
    }
]

## Function calling steps

1. Call model with functions defined (pass as parameter) – along with your system and user messages.
2. Model decides to call which function(s) – model returns the name and input arguments.
3. Execute function code (**on user end**) – parse the model's response and handle function calls.
4. Supply model with results – so it can incorporate them into its final response.
5. Model responds – incorporating the result in its output.

### 1. Define functions & call model

In [70]:
# Functions are defined above under OpenAI Function Schema section

messages = [
    {   "role": "user",
        "content": "How much does 100 USD worth in Eupore? How about in China? What's the highest stock price of AAPL in 2024?"
    }
]

In [71]:
# Quick test to ensure the function works
from openai import OpenAI

# Use OpenAI Compatible API
client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=example_tools,
    tool_choice="auto",
    max_tokens=1000,
)

In [61]:
# Function for printing out responses neatly
def pprint_response(response):
    print("--- Original Response ---\n")
    print(response, "\n")
    
    print("--- Model Response Message ---\n")
    print(response.choices[0].message, "\n")
    
    if response.choices[0].message.tool_calls:
        for i in range(0, len(response.choices[0].message.tool_calls)):
            print(f"--- Tool Call {i+1} ---\n")
            print(f"Function: {response.choices[0].message.tool_calls[i].function.name}\n")
            print(f"Arguments: {response.choices[0].message.tool_calls[i].function.arguments}\n")

### 2. Model decides to call which function(s)

In [72]:
pprint_response(response)

--- Original Response ---

ChatCompletion(id='chatcmpl-Bwh3zhOXqtJSFKuZRwjHt9K6Zc2Qd', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_d0sAg1JoPJ37wEq96D6gTBR1', function=Function(arguments='{"base_currency": "USD", "target_currency": "EUR"}', name='get_exchange_rate'), type='function'), ChatCompletionMessageToolCall(id='call_CYzn4gEHJlaz27eiF0xMxpU4', function=Function(arguments='{"base_currency": "USD", "target_currency": "CNY"}', name='get_exchange_rate'), type='function'), ChatCompletionMessageToolCall(id='call_9fEAb1Hf7FI1XBATxX0JjH0U', function=Function(arguments='{"ticker": "AAPL", "start_date": "2024-01-01", "end_date": "2024-10-31"}', name='fetch_stock_highest_price'), type='function')]))], created=1753328259, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='defa

We can see from the previous response that the model returned three function calling request as `ChatCompletionMessageToolCall`, specifying the function name and arguments extracted from user query. Our next step is to parse these requests and execute the tools on our end.

### 3. Implement and execute functions

In [ ]:
import yfinance as yf
import requests

# Example function to fetch stock data
def fetch_stock_highest_price(ticker, start_date, end_date):
    """
    Fetch historical stock data for a given ticker between start_date and end_date and return the highest stock price.
    
    Args:
        ticker (str): Stock ticker symbol.
        start_date (str): Start date in 'YYYY-MM-DD' format.
        end_date (str): End date in 'YYYY-MM-DD' format.
    
    Returns:
        pd.DataFrame: DataFrame containing stock data.
    """
    data = yf.download(ticker, start=start_date, end=end_date)
    if data.empty:
        raise ValueError(f"No data found for ticker {ticker} between {start_date} and {end_date}.")
    highest_price = data['High'].max().values[0]
    return highest_price

def get_exchange_rate(base_currency, target_currency):
    """
    Fetch the exchange rate between two currencies.
    
    Args:
        base_currency (str): The base currency code (e.g., 'USD').
        target_currency (str): The target currency code (e.g., 'EUR').
    
    Returns:
        float: Exchange rate from base to target currency.
    """
    # Placeholder for actual exchange rate fetching logic
    url = f"https://cdn.jsdelivr.net/npm/@fawazahmed0/currency-api@latest/v1/currencies/{base_currency.lower()}.json"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data.get(base_currency.lower(), {}).get(target_currency.lower(), None)
    else:
        raise Exception(f"Failed to fetch exchange rate for {base_currency}: {response.status_code}")

### 4 & 5. Supply model with results & Model responds

In [ ]:
# Check the entire conversation flow
import inspect # used for function signature inspection

def print_conversation_flow(prompt, tools):

    # 1. Initialize the conversation with the user's prompt
    input_messages = [
        {"role": "user", "content": prompt}
    ]

    print("\nInitial Messages: ", input_messages)

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=input_messages,
        tools=tools,
        tool_choice="auto",
    )

    response_message = response.choices[0].message
    print("\nResponse Message:", response_message)

    # 2. Check if the model called any tools
    tool_calls = response_message.tool_calls
    print("\nTool Calls:", tool_calls)

    if tool_calls:
        # extend conversation with assitant's reply (tool calls)
        input_messages.append(response_message)
        available_functions = {
            "fetch_stock_highest_price": fetch_stock_highest_price,
            "get_exchange_rate": get_exchange_rate
        }
        
        # Step 3. Call the function with the arguments provided by the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions.get(function_name, None) # Get the function callable
            if function_to_call is None:
                print(f"\nWarning: Function {function_name} is not defined in available tools.")
                continue
            arguments = json.loads(tool_call.function.arguments)
            
            func_sig = inspect.signature(function_to_call)
            call_args = {
                k: arguments.get(k, v.default) for k, v in func_sig.parameters.items() if k in arguments or v.default is not inspect.Parameter.empty
            }
            print(f"\nCalling function: {function_name} with arguments: {call_args}")

            function_response = str(function_to_call(**call_args))

            print("\nFunction Response:", function_response)

            # Step 4. Put output into a tool message
            tool_output_message = ({
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response
            })
            print("\nAppending tool Messages:", tool_output_message)

            input_messages.append(tool_output_message)

    # Step 5. Call the model again with tool outputs included
    final_response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=input_messages,
    )

    print("\nFinal Response:", final_response)
    print("\nFormatted Response:", final_response.choices[0].message.content)

    return

# Example usage
prompt = "How much does 100 USD worth in Europe? How about in China? What's the stock price of AAPL in 2024?"
print_conversation_flow(prompt, example_tools)


Initial Messages:  [{'role': 'user', 'content': "How much does 100 USD worth in Europe? How about in China? What's the stock price of AAPL in 2024?"}]


/var/folders/9v/463zyd495kbd_1d2jq0mcvzm0000gn/T/ipykernel_79226/294069534.py:17: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed


Response Message: ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_dV2RIHs8I6sizHQNbcCttcoj', function=Function(arguments='{"base_currency": "USD", "target_currency": "EUR"}', name='get_exchange_rate'), type='function'), ChatCompletionMessageToolCall(id='call_8zzfSs18l1TwLMLTKujwqrDG', function=Function(arguments='{"base_currency": "USD", "target_currency": "CNY"}', name='get_exchange_rate'), type='function'), ChatCompletionMessageToolCall(id='call_IK3hYUa80S6mqkwE8eCEAer2', function=Function(arguments='{"ticker": "AAPL", "start_date": "2024-01-01", "end_date": "2024-12-31"}', name='fetch_stock_highest_price'), type='function')])

Tool Calls: [ChatCompletionMessageToolCall(id='call_dV2RIHs8I6sizHQNbcCttcoj', function=Function(arguments='{"base_currency": "USD", "target_currency": "EUR"}', name='get_exchange_rate'), type='function'), ChatCompletionMessageToolCall(id='ca


Final Response: ChatCompletion(id='chatcmpl-BwhDKX8VTk4zOIw6ArRbLRfU1SCEK', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='As of the latest available exchange rates:\n\n- **100 USD is worth approximately 85.18 EUR** in Europe.\n- **100 USD is worth approximately 716.94 CNY** in China.\n\nRegarding the stock price of Apple Inc. (AAPL) in 2024, it reached a peak price of approximately **$259.47** during that year.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1753328838, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=75, prompt_tokens=164, total_tokens=239, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=

## Function calling behavior (at model side)

There is a parameter `tool_choice` we can specify when calling LLMs with list of tools. This lets the model decide whether to call functions and, if so, which functions to call.

There are four options for this parameter to customize the behavior:
1. Default behavior: Call zero, one or multiple functions. `tool_choise: "auto"`
2. To force the model to **always call one or more functions**, you can set `tool_choice: "required"`. The model will then select which function(s) to call.
3. To force the model to **call only one specific function**, you can set `tool_choice: {"type": "function", "name": "my_specific_function"}`.
4. To **disable function calling** and force the model to imitate the behavior of no functions are passed, you can set `tool_choice: "none"`.